In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature
from azure.ai.documentintelligence.models._models import AnalyzeResult

import pandas as pd
import numpy as np
import functools
import threading
import json
import io
import re
import os
import uuid
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

True

In [87]:
key = os.environ.get('OCR_KEY')
endpoint = os.environ.get('OCR_ENDPOINT')
credential = AzureKeyCredential(key)
document_intelligence_client = DocumentIntelligenceClient(endpoint, credential, api_version='2023-10-31-preview')
doc_path ='../data/german-handwriting-sample.jpeg'

with open(doc_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", 
        analyze_request=f, 
        content_type="application/octet-stream", 
    )
r_cloud = poller.result()

In [5]:
import requests
import json

service_url = 'http://localhost:5000/'

# Define the URL and file path
url = f"{service_url}formrecognizer/documentModels/prebuilt-read:syncAnalyze?api-version=2022-08-31"
file_path = "../data/handwritten-form.jpg"

# Define headers
headers = {
    'accept': '*/*',
    'Content-Type': 'application/octet-stream',
}

# Open the file and send the POST request
with open(file_path, 'rb') as file:
    response = requests.post(url, headers=headers, data=file)

r_local = json.loads(response.text)
r_local = r_local['analyzeResult']

# Inspect Response

In [19]:
print(f'cloud keys: {r_cloud.keys()}')
print(f'local keys: {r_local.keys()}')

cloud keys: dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'contentFormat'])
local keys: dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'languages'])


In [31]:
print(f'cloud keys: {r_cloud["pages"][0].keys()}')
print(f'local keys: {r_local["pages"][0].keys()}')

print(f'cloud keys: {r_cloud["styles"]}')
print(f'local keys: {r_local["styles"]}')

cloud keys: dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans'])
local keys: dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans', 'kind'])
cloud keys: [{'confidence': 1, 'spans': [{'offset': 34, 'length': 6}, {'offset': 56, 'length': 20}, {'offset': 303, 'length': 32}, {'offset': 339, 'length': 2}, {'offset': 346, 'length': 3}, {'offset': 354, 'length': 3}, {'offset': 374, 'length': 5}, {'offset': 387, 'length': 6}, {'offset': 399, 'length': 4}, {'offset': 410, 'length': 5}, {'offset': 423, 'length': 6}, {'offset': 433, 'length': 2}, {'offset': 441, 'length': 4}, {'offset': 452, 'length': 5}, {'offset': 465, 'length': 6}, {'offset': 475, 'length': 2}, {'offset': 482, 'length': 3}, {'offset': 492, 'length': 5}, {'offset': 505, 'length': 9}, {'offset': 519, 'length': 3}, {'offset': 543, 'length': 6}, {'offset': 553, 'length': 2}, {'offset': 560, 'length': 3}, {'offset': 569, 'length': 4}, {'offset': 580, 'length': 5},

In [21]:
print(f'cloud keys: {r_cloud["pages"][0]["words"][0].keys()}')
print(f'local keys: {r_local["pages"][0]["words"][0].keys()}')

cloud keys: dict_keys(['content', 'polygon', 'confidence', 'span'])
local keys: dict_keys(['content', 'polygon', 'confidence', 'span'])


In [27]:
print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["polygon"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["polygon"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["content"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["content"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["confidence"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["confidence"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["span"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["span"]}')

cloud keys: [187, 15, 313, 15, 313, 34, 187, 33]
local keys: [186, 13, 315, 14, 314, 37, 186, 35]
cloud keys: HANDWRITING
local keys: HANDWRITING
cloud keys: 0.993
local keys: 0.991
cloud keys: {'offset': 0, 'length': 11}
local keys: {'offset': 0, 'length': 11}


# Test Parsing Method

In [89]:
text_granularity = 'paragraph'
model_id = 'prebuilt-read'

def parse_ocr_result(result) -> pd.DataFrame:
    text_granularity = 'PARAGRPAH'
    model_id = 'prebuilt-layout'
    parsed_result = None
    # azure doesn't provide results on page level natively
    level = text_granularity
    if (level.upper() == 'PAGE'):
        text_granularity = "LINE"
    else:
        text_granularity = level.upper()

    for page in result['pages']:
        try:
            contains_handwriting = result.styles[0].is_handwritten
        except:
            contains_handwriting = False

        ocr_data = []
        
        # to calculate the average confidence
        if text_granularity != "WORD":
            word_confidences = [word['confidence'] for word in page['words']]
            total_confidence = sum(word_confidences)
            total_words = len(word_confidences)
            average_confidence = total_confidence / total_words if total_words > 0 else 0
            
        # extraction of (natively provided) results 
        if text_granularity == "PARAGRPAH":
            for paragraph_idx, paragraph in enumerate(result['paragraphs']):
                x1, y1, x2, y2, x3, y3, x4, y4 = paragraph['boundingRegions'][0]['polygon']

                try: 
                    role = paragraph['role']
                except:
                    role = ''

                paragrpah_info = {
                    "page": paragraph['boundingRegions'][0]['pageNumber'],
                    "paragraph": paragraph_idx,
                    "text": paragraph['content'],
                    "role": role,
                    "bb_x1": x1,
                    "bb_y1": y1,
                    "bb_x2": x2,
                    "bb_y2": y2,
                    "bb_x3": x3,
                    "bb_y3": y3,
                    "bb_x4": x4,
                    "bb_y4": y4,
                    "offset": paragraph['spans'][0]['offset'],
                    "length": paragraph['spans'][0]['length'],
                    
                }
                
                ocr_data.append(paragrpah_info)

        elif text_granularity == "LINE":
            for line_idx, line in enumerate(page['lines']):
                x1, y1, x2, y2, x3, y3, x4, y4 = line['polygon']

                line_info = {
                    "page": page['pageNumber'],
                    "line": line_idx,
                    "text": line['content'],
                    "bb_x1": x1,
                    "bb_y1": y1,
                    "bb_x2": x2,
                    "bb_y2": y2,
                    "bb_x3": x3,
                    "bb_y3": y3,
                    "bb_x4": x4,
                    "bb_y4": y4,
                    "offset": line['spans'][0]['offset'],
                    "length": line['spans'][0]['length'],
                }
                
                ocr_data.append(line_info)

        elif text_granularity == "WORD":
            for word in page.words:
                x1, y1, x2, y2, x3, y3, x4, y4 = word['polygon']

                word_info = {
                    "page": page['pageNumber'],
                    "text": word['content'],
                    "confidence": word['confidence'],
                    "bb_x1": x1,
                    "bb_y1": y1,
                    "bb_x2": x2,
                    "bb_y2": y2,
                    "bb_x3": x3,
                    "bb_y3": y3,
                    "bb_x4": x4,
                    "bb_y4": y4,
                    "offset": word['span']['offset'],
                    "length": word['span']['length'],
                    }
                
                ocr_data.append(word_info)
        
        df = pd.DataFrame(ocr_data)

        # in case texts should be aggreagted on page level
        if level.upper() == "PAGE":
            ocr_data = []
            page_info = {
                    "page": page['pageNumber'],
                    "text": "\n ".join(df['text']),
                    "avg_confidence": average_confidence,
                    "contains_handwriting": contains_handwriting,
                    "bb_x1": df["bb_x1"].min(),
                    "bb_y1": df["bb_y1"].min(),
                    "bb_x2": df["bb_x2"].max(),
                    "bb_y2": df["bb_y2"].min(),
                    "bb_x3": df["bb_x3"].max(),
                    "bb_y3": df["bb_y3"].max(),
                    "bb_x4": df["bb_x4"].min(),
                    "bb_x4": df["bb_x4"].max(),
                    }
            ocr_data.append(page_info)
            
            df = pd.DataFrame(ocr_data)

    if model_id == 'prebuilt-read' and text_granularity.upper() == 'PARAGRAPH': # 'read' model doesn't provide semantic role, only 'layout' does
        parsed_result = parsed_result.drop(columns=['role'])

    return df


df = parse_ocr_result(r_cloud)
df.head()

,page,paragraph,text,role,bb_x1,bb_y1,bb_x2,bb_y2,bb_x3,bb_y3,bb_x4,bb_y4,offset,length
0,1,0,"Liebe Handschrift,",,89,41,339,36,340,78,90,83,0,18
1,1,1,ich mag Dich nicht besonders. Jeder Drittkläss...,,81,124,917,108,923,399,86,415,19,372
2,1,2,"Aber brauche ich Dich noch in einem Alltag, in...",,87,423,946,438,942,667,83,652,392,319
3,1,3,Du bildest unsere Persönlichkeit ab. Trotzdem ...,,86,696,934,701,932,1056,84,1051,712,432
4,1,4,"Schriftliche grüße ,",,94,1081,338,1087,337,1128,93,1122,1145,20


In [95]:
r_cloud['paragraphs'][3].keys()

dict_keys(['spans', 'boundingRegions', 'content'])

In [51]:
for page in r_cloud['pages']:

    word_confidences = [word['confidence'] for word in page.words]
    print(word_confidences)

[0.993, 0.997, 0.986, 0.865, 0.984, 0.992, 0.984, 0.996, 0.994, 0.918, 0.981, 0.951, 0.97, 0.992, 0.992, 0.993, 0.994, 0.991, 0.991, 0.994, 0.997, 0.998, 0.998, 0.994, 0.993, 0.993, 0.998, 0.993, 0.994, 0.993, 0.997, 0.962, 0.993, 0.992, 0.998, 0.994, 0.994, 0.999, 0.999, 0.994, 0.991, 0.994, 0.988, 0.977, 0.989, 0.994, 0.992, 0.995, 0.951, 0.991, 0.945, 0.998, 0.231, 0.994, 0.924, 0.99, 0.981, 0.995, 0.852, 0.994, 0.99, 0.975, 0.951, 0.997, 0.967, 0.994, 0.974, 0.993, 0.999, 0.991, 0.995, 0.993, 0.997, 0.993, 0.994, 0.998, 0.996, 0.998, 0.997, 0.995, 0.99, 0.994, 0.918, 0.924, 0.998, 0.984, 0.988, 0.844, 0.972, 0.994, 0.994, 0.999, 0.999, 0.998, 0.988, 0.981, 0.963, 0.997, 0.994, 0.878, 0.154, 0.335, 0.859, 0.891, 0.99, 0.993, 0.998, 0.992, 0.988, 0.999, 0.998, 0.976, 0.988, 0.989, 0.998, 0.994, 0.998, 0.998, 0.994, 0.993, 0.99, 0.991, 0.997, 0.993, 0.991, 0.993, 0.989, 0.994, 0.993, 0.994, 0.993, 0.99, 0.959, 0.993, 0.997, 0.993, 0.989, 0.975, 0.994, 0.998, 0.992, 0.988, 0.998, 0.991